In [5]:
!pip install -q tensorflow


In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Load and preprocess dataset (e.g., CIFAR-10)
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# For simplicity, we classify just 3 classes: airplane, car, bird
selected_classes = [0, 1, 2]
mask_train = np.isin(y_train, selected_classes).flatten()
mask_test = np.isin(y_test, selected_classes).flatten()

x_train, y_train = x_train[mask_train], y_train[mask_train]
x_test, y_test = x_test[mask_test], y_test[mask_test]


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [7]:
model = keras.Sequential([
    keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(32,32,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')  # 3 classes
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.6327 - loss: 0.7889 - val_accuracy: 0.7930 - val_loss: 0.5212
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.8054 - loss: 0.4759 - val_accuracy: 0.8290 - val_loss: 0.4389
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8296 - loss: 0.4282 - val_accuracy: 0.8360 - val_loss: 0.4158
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8503 - loss: 0.3899 - val_accuracy: 0.8343 - val_loss: 0.4072
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8506 - loss: 0.3655 - val_accuracy: 0.8373 - val_loss: 0.4230


In [8]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved successfully.")


Saved artifact at '/tmp/tmpdt04x82x'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  133658376763536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133658376762768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133658374885200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133658374884048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133658374884240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133658374885776: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ TFLite model saved successfully.


In [9]:
from google.colab import files
files.download("model.tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>